# Semantic Kernel - MCP integration

## Packages and configurations

In [ ]:
# Copyright (c) Microsoft. All rights reserved.
%pip install --upgrade semantic-kernel
%pip install --upgrade semantic-kernel[mcp]

import asyncio
import os

from typing import Annotated
from dotenv import load_dotenv
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread   
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior



load_dotenv()

## Travel Agent with MCP 🛠️🛠️

In [ ]:

async with (
    MCPStdioPlugin(
        name="Airbnb",
        description="Airbnb mcp server",
        command="npx",
        args=[
            "-y",
            "@openbnb/mcp-server-airbnb",
            "--ignore-robots-txt",
        ],
    ) as airbnb_plugin,
    MCPStdioPlugin(
        name="Ticketmaster",
        description="Ticketmaster mcp server",
        command="npx",
        args=[
            "-y",
            "@delorenj/mcp-server-ticketmaster"
        ],
        env={"TICKETMASTER_API_KEY": os.getenv("TICKETMASTER_API_KEY")},
    ) as ticketmaster_plugin,
):
    
    agentWmcp = ChatCompletionAgent(
        service=AzureChatCompletion(
            deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        ),
        name="TravelAgent",
        instructions=
            """
                You are a travel agent. You can help the user find concerts and properties.
                Appartments from Airbnb must by order by rating and shown only if they are rated 4.80 or higher.
                Estimate the cost for lodging.
                Show appartments first, then concerts.
            """,
        plugins=[airbnb_plugin, ticketmaster_plugin],
        function_choice_behavior=FunctionChoiceBehavior.Auto()
        # function_choice_behavior=FunctionChoiceBehavior.NoneInvoke()
    )

    USER_INPUTS = [
        """
            I am travelling to New York from 10th of June to 20th of June 2025.
            could you please help me find a place to stay and rock concerts to attend?
        """,
    ]

    # Create a thread to hold the conversation
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None
    for user_input in USER_INPUTS:
        print(f"# User: {user_input}", end="\n\n")
        # Invoke the agent for a response
        # user_input = "Show all functions you would execute to get the goal:" + user_input
        response = await agentWmcp.get_response(messages=user_input, thread=thread)
        print(f"# {response.name}: {response} ")
        thread = response.thread

    # Cleanup: Clear the thread
    await thread.delete() if thread else None